<a href="https://colab.research.google.com/github/SeungHwaJung/Auto-Mobile-/blob/main/0718_python_yolov11_yt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Youtube에서 다운로드 후 영상 추론

In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
# 📦 라이브러리 설치
!pip install yt-dlp opencv-python

import os
import cv2
import torch
from ultralytics import YOLO
from IPython.display import display
from google.colab import files

# COCO로 사전 학습된 YOLOv11n 모델 로드
model = YOLO("yolo11n.pt")


# -----------------------------------------------------------------------------
# 🎥 1. 유튜브 영상 다운로드 함수
# -----------------------------------------------------------------------------
def download_youtube_video(youtube_url, output_path='/content'):
    import yt_dlp

    ydl_opts = {
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'merge_output_format': 'mp4',
        'quiet': True
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=True)
        filename = ydl.prepare_filename(info).replace('.webm', '.mp4').replace('.mkv', '.mp4')
        print(f"✅ 다운로드 완료: {filename}")
        return filename

# -----------------------------------------------------------------------------
# 🧠 2. YOLOv11 추론 수행 함수
# -----------------------------------------------------------------------------
def run_yolo_on_video(input_path, output_path='/content/yolo12_result.mp4'):
    cap = cv2.VideoCapture(input_path)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out    = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"🎬 총 프레임: {frame_count} | FPS: {fps}")

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO 추론 및 시각화
        results = model.predict(frame, verbose=False)
        annotated_frame = results[0].plot()

        out.write(annotated_frame)
        frame_idx += 1

        if frame_idx % 30 == 0:
            print(f"🛠 처리 중... {frame_idx}/{frame_count} 프레임")

    cap.release()
    out.release()
    print(f"✅ YOLO 추론 영상 저장 완료: {output_path}")
    return output_path

# -----------------------------------------------------------------------------
# 🚀 3. 전체 실행 함수
# -----------------------------------------------------------------------------
def youtube_to_yolo_inference(youtube_url):
    video_path = download_youtube_video(youtube_url)
    result_path = run_yolo_on_video(video_path)
    files.download(result_path)  # Colab에서 결과 파일 다운로드 버튼 표시

# -----------------------------------------------------------------------------
# ▶️ 실행 예시
# -----------------------------------------------------------------------------
# 유튜브 영상 URL을 아래에 입력해서 실행
youtube_url = "https://www.youtube.com/watch?v=Zl-AgE-8oUg"
youtube_to_yolo_inference(youtube_url)


100%|██████████| 5.34M/5.34M [00:00<00:00, 220MB/s]


✅ 다운로드 완료: /content/주행.mp4
🎬 총 프레임: 1175 | FPS: 24.0
🛠 처리 중... 30/1175 프레임
🛠 처리 중... 60/1175 프레임
🛠 처리 중... 90/1175 프레임
🛠 처리 중... 120/1175 프레임
🛠 처리 중... 150/1175 프레임
🛠 처리 중... 180/1175 프레임
🛠 처리 중... 210/1175 프레임
🛠 처리 중... 240/1175 프레임
🛠 처리 중... 270/1175 프레임
🛠 처리 중... 300/1175 프레임
🛠 처리 중... 330/1175 프레임
🛠 처리 중... 360/1175 프레임
🛠 처리 중... 390/1175 프레임
🛠 처리 중... 420/1175 프레임
🛠 처리 중... 450/1175 프레임
🛠 처리 중... 480/1175 프레임
🛠 처리 중... 510/1175 프레임
🛠 처리 중... 540/1175 프레임
🛠 처리 중... 570/1175 프레임
🛠 처리 중... 600/1175 프레임
🛠 처리 중... 630/1175 프레임
🛠 처리 중... 660/1175 프레임
🛠 처리 중... 690/1175 프레임
🛠 처리 중... 720/1175 프레임
🛠 처리 중... 750/1175 프레임
🛠 처리 중... 780/1175 프레임
🛠 처리 중... 810/1175 프레임
🛠 처리 중... 840/1175 프레임
🛠 처리 중... 870/1175 프레임
🛠 처리 중... 900/1175 프레임
🛠 처리 중... 930/1175 프레임
🛠 처리 중... 960/1175 프레임
🛠 처리 중... 990/1175 프레임
🛠 처리 중... 1020/1175 프레임
🛠 처리 중... 1050/1175 프레임
🛠 처리 중... 1080/1175 프레임
🛠 처리 중... 1110/1175 프레임
🛠 처리 중... 1140/1175 프레임
🛠 처리 중... 1170/1175 프레임
✅ YOLO 추론 영상 저장 완료: /content/yolo12_result.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
import time
import pandas as pd

def measure_fps(model, video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    start_time = time.time()

    processed_frames = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO 추론 (verbose=False로 출력 최소화)
        model.predict(frame, verbose=False)
        processed_frames += 1

    cap.release()
    end_time = time.time()
    elapsed = end_time - start_time
    fps = processed_frames / elapsed if elapsed > 0 else 0
    return fps, elapsed, processed_frames

# YOLO 모델 로드 (예시)
model_v8 = YOLO("yolov8n.pt")
model_v11 = YOLO("yolo11n.pt")

video_path = "/content/your_video.mp4"

fps_v8, time_v8, frames_v8 = measure_fps(model_v8, video_path)
fps_v11, time_v11, frames_v11 = measure_fps(model_v11, video_path)

# 결과 테이블 생성
df = pd.DataFrame({
    "Model": ["YOLOv8n", "YOLOv11n"],
    "Processed Frames": [frames_v8, frames_v11],
    "Elapsed Time (s)": [time_v8, time_v11],
    "FPS": [fps_v8, fps_v11]
})

# Markdown 파일로 저장
md_content = df.to_markdown(index=False)
with open("fps_comparison.md", "w") as f:
    f.write("# YOLO 모델 FPS 비교\n\n")
    f.write(md_content)

print(md_content)

100%|██████████| 6.25M/6.25M [00:00<00:00, 336MB/s]

| Model    |   Processed Frames |   Elapsed Time (s) |   FPS |
|:---------|-------------------:|-------------------:|------:|
| YOLOv8n  |                  0 |        3.8147e-06  |     0 |
| YOLOv11n |                  0 |        1.19209e-06 |     0 |
